# Voldemort's Stable Diffusion WebUI

Adapted from: https://colab.research.google.com/drive/1Iy-xW9t1-OQWhb0hNxueGij8phCyluOh

In the Tools section at the bottom of the notebook there is a block to download the latest version of this notebook from Github.

**Guides**
- [Getting started on Paperspace](https://github.com/Engineer-of-Stuff/stable-diffusion-paperspace/blob/main/Docs/Paperspace%20Guide%20for%20Retards.md)
- [Using the WebUI](https://rentry.org/voldy)
- [Using the Inpainter](https://rentry.org/drfar)
- [Textual Inversion](https://rentry.org/aikgx)
- [Crowd-Sourced Prompts](https://lexica.art/)
- [Artist Name Prompts](https://sgreens.notion.site/sgreens/4ca6f4e229e24da6845b6d49e6b08ae7?v=fdf861d1c65d456e98904fe3f3670bd3)

## Installation and Setup

You must reinstall everything each time you restart the machine. If already downloaded dependencies will be auto-updated.

**Where to store the models**

`/storage/models/` is for the free tier.

`/notebooks/models/` is for the pro tier.

_You must run the block below or else the variable won't be accessable in the notebook._

In [ ]:
model_storage_dir = '/storage/models' # no trailing slash

# ===============================================================
# Save the variable to Jupiter's temp storage to access it when the kernel restarts.
# To reset your storage directory, rerun this cell.
%store model_storage_dir

### Clone the central repository

In [ ]:
import os.path
%store -r model_storage_dir
if not os.path.exists('/notebooks/stable-diffusion-webui'):
    %cd /notebooks/
    !git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui
else: # update repo if already exists
    print('stable-diffusion-webui already downloaded, updating')
    %cd /notebooks/stable-diffusion-webui
    !git pull

### Install requirements and download repositories.

In [ ]:
%cd /notebooks/stable-diffusion-webui
%store -r model_storage_dir
!pip install -r requirements.txt

if not os.path.exists('/notebooks/stable-diffusion-webui/repositories'): # download repositories if they don't exist
    !mkdir /notebooks/stable-diffusion-webui/repositories
    !git clone https://github.com/CompVis/stable-diffusion.git /notebooks/stable-diffusion-webui/repositories/stable-diffusion
    !git clone https://github.com/CompVis/taming-transformers.git /notebooks/stable-diffusion-webui/repositories/taming-transformers
    !git clone https://github.com/sczhou/CodeFormer.git /notebooks/stable-diffusion-webui/repositories/CodeFormer
    !git clone https://github.com/salesforce/BLIP.git /notebooks/stable-diffusion-webui/repositories/BLIP
else: # update repositories if they do exist
    print('Updating dependencies')
    for dir in os.listdir('/notebooks/stable-diffusion-webui/repositories'):
        %cd /notebooks/stable-diffusion-webui/repositories/$dir
        !git pull
    
!pip install -r /notebooks/stable-diffusion-webui/repositories/CodeFormer/requirements.txt

if not os.path.exists('/notebooks/stable-diffusion-webui/GFPGANv1.3.pth'):
    !wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth
else:
    print('GFPGANv1.3.pth already downloaded')
    
!mkdir -p $model_storage_dir

# Get some storage back
!pip cache purge
!sudo apt-get clean

### Download the model

I've provided a few different ways of aquiring the models. Try the torrent option first.

You don't need to repeat this step if you've already downloaded the models.

**Filesize and Storage Disclaimer**

Paperspace free tier has only 5GB of storage space. The standard model is 4GB and waifu is 3.5GB/7.2GB. To get around this limited storage capacity we will download the models to `/storage/`, a non-persistent directory. Each time you restart the machine you will have to redownload the models.

If you don't want to have to redownload the models every time you restart your machine you have three options:
1. Add a payment method to your account. Storage overages are billed at \$0.29/GB and billing occurs monthly and runs at midnight on the first of each month. With a payment method on file, Paperspace will let you use more storage and if you time it right you shouldn't actually be charged for it.
2. Upgrade to a Pro account. They'll give you 15GB and you'll get longer runtimes and more powerful free GPUs.
3. Create an account using my referral code `KQLRH37` or [signup link](https://console.paperspace.com/signup?R=KQLRH37). You'll get \$10 credit that you should be able to put towards the storage overage charges. If you've already created your account you can still redeem the code at the bottom of the Billing page.

#### Standard Model

**Torrent**

In [ ]:
%store -r model_storage_dir
!apt update
!apt install -y aria2
%cd $model_storage_dir
!aria2c --seed-time=0 --max-overall-upload-limit=1K "magnet:?xt=urn:btih:3A4A612D75ED088EA542ACAC52F9F45987488D1C&tr=udp://tracker.opentrackr.org:1337"

**Web Download**

Voldy provided an alternative download if you don't want to use HuggingFace.

[https://drive.google.com/file/d/1wHFgl0ivCmIZv88hVZXkb8oy9qCuaBGA/view](https://drive.google.com/file/d/1wHFgl0ivCmIZv88hVZXkb8oy9qCuaBGA/view)

Download it to your computer then upload it to your model storage directory (make sure it's named `sd-v1-4.ckpt`).

HuggingFace is much faster and reliable but you need to get access to the repo and provide your user token.

In [ ]:
user_token = "<enter your user token here>"

# ===============================================================================================
%store -r model_storage_dir
!wget --header="'Authorization: Bearer {user_token}'" https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4.ckpt -O $model_storage_dir/sd-v1-4.ckpt

#### Waifu Diffusion

**Torrent**

In [ ]:
!apt update
!apt install -y aria2
%cd $model_storage_dir
!aria2c --seed-time=0 --max-overall-upload-limit=1K "magnet:?xt=urn:btih:F45CECF4E9DE86DA83A78DD2CCCD7F27D5557A52&tr=udp://nyquist.localghost.org:6969"

**Web Download**

Very slow

In [ ]:
%store -r model_storage_dir
!wget https://thisanimedoesnotexist.ai/downloads/wd-v1-2-full-ema.ckpt -O $model_storage_dir/wd-v1-2-full-ema.ckpt

**Half-Size Model**

If you're on free tier try this first, it's 3.5GB.

In [ ]:
%store -r model_storage_dir
!apt update
!apt install -y aria2
%cd $model_storage_dir
!aria2c --seed-time=0 --max-overall-upload-limit=1K "magnet:?xt=urn:btih:153590FD7E93EE11D8DB951451056C362E3A9150&dn=wd-v1-2-full-ema-pruned.ckpt&tr=udp://glotorrents.pw:6969/announce&tr=udp://tracker.opentrackr.org:1337/announce&tr=udp://torrent.gresille.org:80/announce&tr=udp://tracker.openbittorrent.com:80&tr=udp://tracker.coppersurfer.tk:6969&tr=udp://tracker.leechers-paradise.org:6969&tr=udp://p4p.arenabg.ch:1337&tr=udp://tracker.internetwarriors.net:1337&tr=wss://tracker.openwebtorrent.com"

#### Gradient Dataset Import

Paperspace provides a way to mount external storage to notebooks, called *Datasets*. This is an advanced method but it will save you many GB since you can reuse the same model file across different notebooks.

https://docs.paperspace.com/gradient/cli/datasets

In [ ]:
mounted_dataset_path = '/datasets/<name>'

# ============================================================
%store -r model_storage_dir
import os
for file in os.listdir(mounted_dataset_path):
    if file.endswith("ckpt"):
        !ln -s $mounted_dataset_path/$file $model_storage_dir/$file
        !ls -la --block-size=GB $model_storage_dir/$file

### Clean up and restart the kernel

In [ ]:
%store -r model_storage_dir
!rm $model_storage_dir/*.aria2
!apt autoremove --purge -y aria2
import os
os.kill(os.getpid(), 9)

# Launch the WebUI

First, enter the name of the model you want to use into `model_name` in the cell below.

Then run the model. You will get a link to nnn.gradio.app, that's your WebUI. Follow it.

If you have any issues, restart the kernel.

**Choose your Model**
|Name||
|-|-|
|`sd-v1-4.ckpt`|Standard diffusion v1.4|
|`wd-v1-2-full-ema.ckpt`|waifu-diffusion v1.2|
|`wd-v1-2-full-ema-pruned.ckpt`|waifu-diffusion v1.2 half-size|

Enter the name of the model you want to use in `model_name` below.

To change models you must restart the kernel for it to take effect.

**Start the WebUI**

In [ ]:
model_name = ''

# ===============================================================================================================
# Don't change anything in here
%store -r model_storage_dir
model_file = f'{model_storage_dir}/{model_name}'

import sys
import os

if not os.path.exists(model_file):
    print('Cannot find model file:', model_file)
    sys.exit(1)

if model_name == '':
    print('You forgot to fill out the variable model_name above.')
    sys.exit(1)

# ===============================================================================================================
    
# You can modify the args if nessesary
!python /notebooks/stable-diffusion-webui/webui.py --share --ckpt $model_file # --opt-split-attention

# Export Generations

This block will rename and compress the outputs with 7zip max compression. It expects you to have `log/` and `outputs/` in `/notebooks/`.

In [ ]:
!apt update
!apt install -y p7zip-full
from datetime import datetime
import os
datetime_str = datetime.now().strftime('%m-%d-%Y_%H:%M:%S')
%cd /notebooks/
!mkdir $datetime_str
!mv log $datetime_str
!mv outputs $datetime_str
print('Compressing, please wait...')
os.system(f'7z a -t7z -m0=lzma2 -mx=9 -mfb=64 -md=32m -ms=on {datetime_str}.7z {datetime_str}/')

### Delete old output folder

This block will delete the folder you just compressed.

In [ ]:
!rm -rf /notebooks/$datetime_str/
!echo Deleted /notebooks/$datetime_str/

# Tools

**Show graphics card info**

In [ ]:
!nvidia-smi

**Download the latest version of this notebook from Github**

Run this and refresh the page (press F5). Don't save anything.

In [ ]:
!mv /notebooks/StableDiffusionUI_Voldemort_paperspace.ipynb /notebooks/StableDiffusionUI_Voldemort_paperspace.ipynb.backup
!wget https://raw.githubusercontent.com/Engineer-of-Stuff/stable-diffusion-paperspace/main/StableDiffusionUI_Voldemort_paperspace.ipynb -O /notebooks/StableDiffusionUI_Voldemort_paperspace.ipynb